In [1]:
#step1
import pandas as pd
import numpy as np
egfr = pd.read_csv('All_egfr.csv')
egfr = egfr[(egfr['date'] > '2009-12-31') & (egfr['date'] <= '2019-01-01')]
egfr_acct = list(set(list(egfr['TMA_Acct'])))
all_egfr = pd.read_csv('All_egfr.csv')
all_egfr = all_egfr[all_egfr['TMA_Acct'].isin(egfr_acct)]
def num(df):
    print(len(df))
    print(len(df['TMA_Acct'].unique()))
def show(df, n):
    print(pd.DataFrame(df[df['TMA_Acct'] == n]))
def mapping(n):
    if n >= 90.0: stage = '1'
    elif n >= 60.0: stage = '2'
    elif n >= 45.0: stage = '3a'
    elif n >= 30.0: stage = '3b'
    elif n >= 15.0: stage = '4'
    else: stage = '5'
    return(stage)
num(egfr)
num(all_egfr)

71322
7004
90812
7004


In [2]:
early = pd.DataFrame(all_egfr.groupby(['TMA_Acct']).agg({'date': [np.min]})).reset_index()
early.columns=['TMA_Acct', 'min']
num(early)
early.head(2)

7004
7004


,TMA_Acct,min
0,37,2009-03-25
1,64,2018-05-15 00:00:00


### enter after 2010

In [3]:
enter_after_2010 = early[(early['min'] > '2009-12-31') & (early['min'] <= '2019-01-01')]
num(enter_after_2010)

4702
4702


In [4]:
first_enter_after_2010 = pd.merge(egfr, enter_after_2010, how = 'inner', left_on = ['TMA_Acct', 'date'], right_on = ['TMA_Acct', 'min'])
num(first_enter_after_2010)

4856
4702


In [5]:
first_enter_after_2010_egfr = pd.DataFrame(first_enter_after_2010[['TMA_Acct', 'egfr']].groupby(['TMA_Acct']).agg({'egfr': np.mean})).reset_index()

In [6]:
first_enter_after_2010_egfr['stage'] = first_enter_after_2010_egfr['egfr'].apply(lambda x: mapping(x))

In [7]:
first_enter_after_2010_date = first_enter_after_2010[['TMA_Acct', 'date']]
first_enter_after_2010_date = first_enter_after_2010_date.drop_duplicates()
num(first_enter_after_2010_date)

4702
4702


In [8]:
first_enter_after_2010_stage = pd.merge(first_enter_after_2010_egfr, first_enter_after_2010_date, how = 'inner', left_on = 'TMA_Acct', right_on = 'TMA_Acct')

In [9]:
num(first_enter_after_2010_stage)

4702
4702


In [10]:
first_enter_after_2010_stage.rename(columns={"date": "min"}, inplace = 'True')
first_enter_after_2010_stage = first_enter_after_2010_stage[['TMA_Acct', 'min', 'egfr', 'stage']]
first_enter_after_2010_stage.head(2)

,TMA_Acct,min,egfr,stage
0,64,2018-05-15 00:00:00,10.000000,5
1,86,2010-10-09,20.121101,4


### enter before 2010

In [11]:
acct_enter_after_2010 = list(first_enter_after_2010_stage['TMA_Acct'])
acct_enter_before_2010 = list(egfr[~egfr['TMA_Acct'].isin(acct_enter_after_2010)]['TMA_Acct'])

In [12]:
all_egfr_enter_before_2010 = all_egfr[all_egfr['TMA_Acct'].isin(acct_enter_before_2010)]
num(all_egfr_enter_before_2010)

47661
2302


In [13]:
def extract_egfr(all_data, time1, time2):
    egfr_1 = all_data[(all_data['date'] > time1) & (all_egfr_enter_before_2010['date'] < time2)]
    early_1 = pd.DataFrame(egfr_1.groupby(['TMA_Acct']).agg({'date': [np.min]})).reset_index()
    early_1.columns=['TMA_Acct', 'min']
    early_1_all = pd.merge(all_data, early_1, how = 'inner', left_on = ['TMA_Acct', 'date'], right_on = ['TMA_Acct', 'min'])
    early_1_all = pd.DataFrame(early_1_all[['TMA_Acct', 'egfr']].groupby(['TMA_Acct']).agg({'egfr': np.mean})).reset_index()
    early_1_all['stage'] = early_1_all['egfr'].apply(lambda x: mapping(x))
    part2 = pd.merge(early_1, early_1_all, how = 'inner', left_on = 'TMA_Acct', right_on = 'TMA_Acct')
    part2_acct = list(part2['TMA_Acct'])
    all_data = all_data[~all_data['TMA_Acct'].isin(part2_acct)]
    num(all_data)
    num(part2)
    return all_data, part2

In [14]:
all_egfr_enter_before_2010, part2 = extract_egfr(all_egfr_enter_before_2010, '2009-12-31', '2010-04-01')

24334
1554
748
748


In [15]:
all_egfr_enter_before_2010, part3 = extract_egfr(all_egfr_enter_before_2010, '2009-09-30', '2010-01-01')

14923
1147
407
407


In [16]:
all_egfr_enter_before_2010, part4 = extract_egfr(all_egfr_enter_before_2010, '2010-03-31', '2010-07-01')

11436
970
177
177


In [17]:
all_egfr_enter_before_2010, part5 = extract_egfr(all_egfr_enter_before_2010, '2009-06-30', '2009-10-01')

8844
830
140
140


In [18]:
all_egfr_enter_before_2010, part6 = extract_egfr(all_egfr_enter_before_2010, '2010-06-30', '2010-10-01')

7742
767
63
63


In [19]:
all_egfr_enter_before_2010, part7 = extract_egfr(all_egfr_enter_before_2010, '2009-03-31', '2009-07-01')

6841
721
46
46


In [20]:
all_egfr_enter_before_2010, part8 = extract_egfr(all_egfr_enter_before_2010, '2010-09-30', '2011-01-01')

6343
698
23
23


In [21]:
all_egfr_enter_before_2010, part9 = extract_egfr(all_egfr_enter_before_2010, '2008-12-31', '2009-04-01')

6162
686
12
12


In [22]:
part10 = all_egfr_enter_before_2010[all_egfr_enter_before_2010['date'] > '2010-01-01']
num(part10)

4426
686


In [23]:
early_10 = pd.DataFrame(part10.groupby(['TMA_Acct']).agg({'date': [np.min]})).reset_index()
early_10.columns=['TMA_Acct', 'min']
num(early_10)
early_10.head(2)

686
686


,TMA_Acct,min
0,116,2011-05-19
1,403,2015-07-23


In [24]:
part10 = pd.merge(part10, early_10, how = 'inner', left_on = ['TMA_Acct', 'date'], right_on = ['TMA_Acct', 'min'])
early_10_all = pd.DataFrame(part10[['TMA_Acct', 'egfr']].groupby(['TMA_Acct']).agg({'egfr': np.mean})).reset_index()
early_10_all['stage'] = early_10_all['egfr'].apply(lambda x: mapping(x))
part10 = pd.merge(early_10, early_10_all, how = 'inner', left_on = 'TMA_Acct', right_on = 'TMA_Acct')
num(part10)

686
686


In [25]:
combine_part = pd.concat([part2, part3, part4, part5, part6, part7, part8, part9, part10], axis = 0)

In [26]:
egfr_earliest = pd.concat([first_enter_after_2010_stage, combine_part], axis = 0)

In [27]:
num(egfr_earliest)

7004
7004


In [28]:
egfr_earliest.to_csv('egfr_earliest.csv')

In [2]:
#step2
egfr_earliest = pd.read_csv('egfr_earliest.csv')
num(egfr_earliest)
egfr_earliest_3a = egfr_earliest[egfr_earliest['stage'] == '3a']
egfr_earliest_3a_acct = list(set(list(egfr_earliest_3a['TMA_Acct'])))
print(len(egfr_earliest_3a))
egfr_earliest_3b = egfr_earliest[egfr_earliest['stage'] == '3b']
egfr_earliest_3b_acct = list(set(list(egfr_earliest_3b['TMA_Acct'])))
egfr_earliest_4 = egfr_earliest[egfr_earliest['stage'] == '4']
egfr_earliest_4_acct = list(set(list(egfr_earliest_4['TMA_Acct'])))
egfr_earliest_5 = egfr_earliest[egfr_earliest['stage'] == '5']
egfr_earliest_5_acct = list(set(list(egfr_earliest_5['TMA_Acct'])))
print(len(egfr_earliest_3b))
print(len(egfr_earliest_4))
print(len(egfr_earliest_5))

7004
7004
1672
1790
1006
555


In [3]:
def final_in_stage(All_egfr_10, stage_acct):
    All_egfr_3a = All_egfr_10[All_egfr_10['TMA_Acct'].isin(stage_acct)]
    All_egfr_3a = All_egfr_3a[['TMA_Acct', 'date', 'stage','egfr']]
    All_egfr_3a = All_egfr_3a.drop_duplicates()
    first = pd.DataFrame(All_egfr_3a.groupby(['TMA_Acct']).agg({'date': [np.min]})).reset_index()
    first.columns=['TMA_Acct', 'min']
    All_egfr_3a = pd.merge(All_egfr_3a, first, how = 'left', left_on = ['TMA_Acct'], right_on = ['TMA_Acct'])
    All_egfr_3a['date'] = pd.to_datetime(All_egfr_3a['date'])
    All_egfr_3a['min'] = pd.to_datetime(All_egfr_3a['min'])
    All_egfr_3a['period'] = ((All_egfr_3a['date'] - All_egfr_3a['min']).dt.days / 90).astype('int')
    egfr_traj = All_egfr_3a[['TMA_Acct', 'egfr', 'period']]
    egfr_traj = pd.DataFrame(egfr_traj.groupby(['TMA_Acct', 'period']).agg({'egfr': np.mean})).reset_index()
    egfr_traj = egfr_traj.pivot(index='TMA_Acct', columns='period', values='egfr')
    print(len(egfr_traj))
    final = egfr_traj.dropna(thresh=3)
    print(len(final))
    for i in range(37):
        final['t' + str(i)] = i
        final.rename(columns={i: "v" + str(i)}, inplace = 'True')
    return final

In [4]:
final_3a = final_in_stage(egfr, egfr_earliest_3a_acct)
final_3b = final_in_stage(egfr, egfr_earliest_3b_acct)
final_4 = final_in_stage(egfr, egfr_earliest_4_acct)
final_5 = final_in_stage(egfr, egfr_earliest_5_acct)

1672
1035


C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


1790
1184
1006
634
555
317


In [5]:
final_3a = final_3a.replace([np.inf, -np.inf], np.nan)
final_3b = final_3b.replace([np.inf, -np.inf], np.nan)
final_4 = final_4.replace([np.inf, -np.inf], np.nan)
final_5 = final_5.replace([np.inf, -np.inf], np.nan)

In [6]:
final_3a.to_csv('final_3a_3.csv')
final_3b.to_csv('final_3b_3.csv')
final_4.to_csv('final_4_3.csv')
final_5.to_csv('final_5_3.csv')

### Trajactory Analysis

In [7]:
baseline = pd.read_csv('baseline.csv')
baseline.head(2)

,Unnamed: 0,TMA_Acct,DOB,Sex,Race,hyper_date,diabe_date,earliest_date_in_all_lab,Date_bmi,BMI,Age_in_2010,avg_egfr,earliest_egfr_date,stage,Patient_Status_Date,Patient_Status
0,0,9,1921,Male,White,2001-05-29,NaN,2000-09-22,NaN,NaN,89,NaN,NaN,NaN,6-30-2010,Deceased
1,1,10,1926,Female,NaN,NaN,NaN,NaN,NaN,NaN,84,NaN,NaN,NaN,NaN,Active


In [10]:
final3a = pd.read_csv('final_3a_3.csv')
final3b = pd.read_csv('final_3b_3.csv')
final4 = pd.read_csv('final_4_3.csv')
final5 = pd.read_csv('final_5_3.csv')

In [11]:
len(final3a)

1287

In [12]:
final3a_1287 = set(list(final3a['TMA_Acct']))

In [13]:
final3a_acct = list(final3a['TMA_Acct'])

In [8]:
def final_base(final, base):
    base = base.fillna('n')
    base = base[['TMA_Acct', 'Sex', 'Race', 'hyper_date', 'diabe_date', 'Age_in_2010']]
    final_base = pd.merge(final, base, how = 'inner', left_on = 'TMA_Acct', right_on = 'TMA_Acct')
    final_base['sex_v'] = final_base['Sex'].apply(lambda x: 1 if x == 'Female' else 0)
    final_base['race_v'] = final_base['Race'].apply(lambda x: 1 if x == 'Black / African American, Black' or x == 'Black/African American' or x == 'Black/African American, Other' else 0)
    final_base['hyper_v'] = final_base['hyper_date'].apply(lambda x: 0 if x == 'n' else 1)
    final_base['diabe_v'] = final_base['diabe_date'].apply(lambda x: 0 if x == 'n' else 1)
    return final_base

In [11]:
final_3a_base = final_base(final3a, baseline)
final_3a_base.to_csv('final_3a_base_3.csv')
final_3b_base = final_base(final3b, baseline)
final_3b_base.to_csv('final_3b_base_3.csv')
final_4_base = final_base(final4, baseline)
final_4_base.to_csv('final_4_base_3.csv')
final_5_base = final_base(final5, baseline)
final_5_base.to_csv('final_5_base_3.csv')

In [12]:
len(final_3a_base)

937

In [19]:
final3a_1085 = set(list(final_3a_base['TMA_Acct']))

In [20]:
missing = [i for i in final3a_1287 if i not in final3a_1085]

In [69]:
master = pd.read_csv("master.csv")

In [72]:
master.head(2)

,Unnamed: 0,TMA_Acct,Account_Date,Sex,DOB,Marital_Status,Race,Location,Zip_Code,Patient_Status_Date,Patient_Status,Ethnicity,Language
0,0,9,06-01-2009,Male,20-04-1921,NaN,White,Hopewell Office,15001,6-30-2010,Deceased,NaN,NaN
1,1,10,06-01-2009,Female,18-05-1926,NaN,NaN,Tma,15360,NaN,Active,NaN,NaN


In [74]:
missing

[18477,
 32816,
 41026,
 47175,
 47195,
 32865,
 45152,
 47206,
 16500,
 30941,
 45277,
 45304,
 33039,
 10550,
 43327,
 39249,
 16737,
 43363,
 37255,
 450,
 45509,
 47559,
 45528,
 18924,
 39405,
 8696,
 10804,
 27199,
 45634,
 35399,
 10863,
 39536,
 41600,
 47764,
 39617,
 39622,
 35533,
 33514,
 43747,
 39677,
 45863,
 47916,
 15156,
 21316,
 33634,
 15204,
 45924,
 45952,
 45969,
 45970,
 35835,
 42027,
 48240,
 46221,
 38052,
 44204,
 36043,
 46292,
 46300,
 40237,
 42287,
 9580,
 36208,
 46466,
 44419,
 15750,
 46470,
 32151,
 28069,
 44452,
 38326,
 46518,
 44531,
 36353,
 40530,
 30292,
 46696,
 48748,
 34414,
 36493,
 44693,
 16027,
 44795,
 34572,
 44831,
 46885,
 40781,
 34653,
 16227,
 44939,
 44953,
 40870,
 44965,
 36783,
 45000,
 16335,
 42988,
 47084]

In [76]:
master[master['TMA_Acct'] == 47084]

,Unnamed: 0,TMA_Acct,Account_Date,Sex,DOB,Marital_Status,Race,Location,Zip_Code,Patient_Status_Date,Patient_Status,Ethnicity,Language


In [77]:
final_3a.head(10)

period,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36
TMA_Acct,,,,,,,,,,,,,,,,,,,,,
97,54.680604,NaN,NaN,NaN,39.538942,33.257395,NaN,NaN,30.545747,NaN,...,27,28,29,30,31,32,33,34,35,36
450,49.000000,NaN,53.0,NaN,49.000000,NaN,53.000000,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
499,54.809306,NaN,NaN,NaN,NaN,40.110099,45.415715,40.735367,44.084108,NaN,...,27,28,29,30,31,32,33,34,35,36
757,55.365724,NaN,NaN,NaN,57.571767,NaN,NaN,NaN,61.755224,NaN,...,27,28,29,30,31,32,33,34,35,36
1025,54.480110,NaN,NaN,65.181731,52.793390,NaN,NaN,NaN,47.236767,NaN,...,27,28,29,30,31,32,33,34,35,36
1052,46.323220,NaN,NaN,NaN,58.169045,NaN,NaN,NaN,53.139278,NaN,...,27,28,29,30,31,32,33,34,35,36
1088,52.793390,NaN,NaN,NaN,59.772795,NaN,NaN,NaN,47.026582,NaN,...,27,28,29,30,31,32,33,34,35,36
1150,51.000000,28.0,29.5,34.527328,NaN,NaN,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
1212,51.199423,NaN,NaN,46.392252,49.000000,NaN,NaN,NaN,NaN,66.0,...,27,28,29,30,31,32,33,34,35,36


In [81]:
final3b_v6 = final_3b[['v0', 'v6']]
final3b_v6.dtypes

period
v0    float64
v6    float64
dtype: object